## Extracció de freqüències

Aquest codi pren les dades recopilades dels partits jugats i en fa un fitxer amb els rendiments per equips. Aquest fitxer és una `xarray` de dimensions `player` vs. `teammate`. És a dir, tenim informació de cada possible combinació d'equips.

In [1]:
import asyncio
import sys

if sys.platform.startswith('win'):
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

In [2]:
# Importem les llibreries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr # per guardar les dades 3D
from collections import Counter

In [3]:
# Carreguem les dades
data_df = pd.read_csv('results.csv')

# Emplenem els espais en blanc amb 0
data_df = data_df.fillna(0.)

In [4]:
# Read the number of games played per each player
dataarray = xr.open_dataset('stats.nc', engine='scipy')
played_games = dataarray['GamesPlayed'].isel(matchday=-1)

In [5]:
# Obtenim una llista amb tots els noms dels participants
players_names = np.unique(data_df[['Jugador 1', 'Jugador 2', 'Jugador 3', 'Jugador 4']].values.flatten())

# Llista de dies jugats
matchdays = pd.unique(data_df['D'])

Crearem les matrius de `xarray` a partir de `pd.DataFrame()`. Per crear els dataframes, iniciarem els diccionaris buits on hi guardarem les dades per cada jugador. Això serà una de les dimensions de la matriu. Després, per cada partit, escriurem al diccionari el paràmetre corresponent a l'alineació. (ESCRIURE MILLOR AIXÒ)

In [6]:
# DataFrame with the number of team occurrences
mates_df = pd.DataFrame(columns = players_names) # number of games played with each mate
matesplayed_df = pd.DataFrame(columns = players_names) # number of games played with each mate divided by number of games playe by player
winmates_df = pd.DataFrame(columns = players_names) # number of games won by each mate
mates_attack_df = pd.DataFrame(columns = players_names) # number of games played with the mate playing as defender (player in attack)
winmates_attack_df = pd.DataFrame(columns = players_names) # number of games won with the mate playing as defender (player in attack)
closematches_df = pd.DataFrame(columns = players_names) # number of games where the game ended 2-3 or 3-2 (won or lost)
closewins_df = pd.DataFrame(columns = players_names) # number of games won by 1 goal (3-2 or 2-3)
closewinsplayed_df = pd.DataFrame(columns = players_names) # number of games won by 1 goal divided by total number of close matches

# Possible lineups
lineups = [['Jugador 1', 'Jugador 2'], ['Jugador 2', 'Jugador 1'], ['Jugador 3', 'Jugador 4'], ['Jugador 4', 'Jugador 3']]

for player in players_names: # for eack player
    # Initialize a dictionary where the number of team-ups will be stored
    mate_count = {}
    mateplayed_count = {} # number of games played with mate divided by total games played
    winmate_count = {}
    mates_attack_count = {}
    winmates_attack_count = {}
    closematches_count = {}
    closewins_count = {}
    closewinsplayed_count = {}

    # Comptadors a 0
    for mate in players_names:
        mate_count[mate] = 0 # set counter to 0
        mateplayed_count[mate] = 0
        winmate_count[mate] = 0
        mates_attack_count[mate] = 0
        winmates_attack_count[mate] = 0
        closematches_count[mate] = 0
        closewins_count[mate] = 0
        closewinsplayed_count[mate] = 0

    # Calculate number of playing counts and victories
    for lineup in lineups:
        lineup_df = data_df[data_df[lineup[0]] == player] # pick games when player plays in position lineup[0]
        teammate_list = lineup_df[lineup[1]] # pick teammate name when player plays in position lineup[1]
        teammate_list_counts = teammate_list.value_counts() # count how many team occurences (teammate in position lineup[1])

        for mate in teammate_list_counts.keys():
            close_victories = 0 # initialize close victories with this mate and this lineup
            close_losses = 0

            mate_count[mate] = mate_count.get(mate, 0) + teammate_list_counts.loc[mate] # store the teammate join number

            mask_mate = (teammate_list == mate) # mask selecting the games that involve 'mate'

            # For the overall games (regardless of position)
            if lineup in [lineups[0], lineups[1]]: # pick lineups playing as local
                number_victories = (lineup_df['Local'][mask_mate] > lineup_df['Visitant'][mask_mate]).sum() # calculate number of victories

                if number_victories: # if the game was won
                    winmate_count[mate] = winmate_count.get(mate, 0) + number_victories # add number of victories

                    if lineup == lineups[1]: # if the game was won with player playing as attacker
                        winmates_attack_count[mate] = winmates_attack_count.get(mate, 0) + number_victories

                close_victories += ((lineup_df['Local'][mask_mate] - lineup_df['Visitant'][mask_mate]) == 1).sum() # calculate number of close victories (3-2)
                close_losses += ((lineup_df['Visitant'][mask_mate] - lineup_df['Local'][mask_mate]) == 1).sum() # calculate number of close losses (2-3)

                # Això si volem distingir victòries com a local/visitant, o bé atacant/defensor
                #closewins_count[mate] = closewins_count.get(mate, 0) + close_victories # store close victories with this mate
                #closematches_count[mate] = closematches_count.get(mate, 0) + (close_victories + close_losses) # store close victories and losses with this mate

                if lineup == lineups[1]: # if the game lineup had the player as attacker
                    mates_attack_count[mate] = mates_attack_count.get(mate, 0) + teammate_list_counts.loc[mate] # games won being attacker, with mate being defender

            elif lineup in [lineups[2], lineups[3]]: # pick lineups playing as visitor    
                if lineup == lineups[3]: # if the game lineup had the player as attacker
                    mates_attack_count[mate] = mates_attack_count.get(mate, 0) + teammate_list_counts.loc[mate] # store occurrences
                number_victories = (lineup_df['Local'][mask_mate] < lineup_df['Visitant'][mask_mate]).sum() # calculate number of victories
                if number_victories: # if the game was won
                    winmate_count[mate] = winmate_count.get(mate, 0) + number_victories # add number of victories
                   
                    if lineup == lineups[3]: # if the game was won with player playing as attacker
                        winmates_attack_count[mate] = winmates_attack_count.get(mate, 0) + number_victories

                close_victories += ((lineup_df['Visitant'][mask_mate] - lineup_df['Local'][mask_mate]) == 1).sum() # calculate number of close victories (3-2)
                close_losses += ((lineup_df['Local'][mask_mate] - lineup_df['Visitant'][mask_mate]) == 1).sum() # calculate number of close losses (2-3)

            closewins_count[mate] = closewins_count.get(mate, 0) + close_victories # store close victories with this mate
            closematches_count[mate] = closematches_count.get(mate, 0) + (close_victories + close_losses) # store close victories and losses with this mate

            # If we take into acount player position
            #if lineup in [lineups[0], lineups[2]]: # pick lineups playing as defender
 #               number_victories = lineup_df[

                
    # Divide count by number of games played by player
    for mate in mate_count.keys():
        # si no hi ha hagut close matches, marquem els valors a -99
        if closematches_count[mate] == 0:
            closematches_count[mate] = np.nan
            closewins_count[mate] = np.nan
            closewinsplayed_count[mate] = np.nan
        if mate_count[mate] == 0: # si no s'ha jugat cap partit amb mate
            mate_count[mate] = np.nan
            mateplayed_count[mate] = np.nan
            winmate_count[mate] = np.nan

        if played_games.sel(player = player).values == 0:
            continue
        else:
            mateplayed_count[mate] = mate_count.get(mate, 0) / played_games.sel(player = player).values
            if closematches_count.get(mate, 0) == 0:
                continue
            else:
                closewinsplayed_count[mate] = closewins_count.get(mate, 0) / closematches_count.get(mate, 0)
    # Append this player result to the overall property dataframe
    mates_df = pd.concat([mates_df, pd.DataFrame([mate_count])])
    matesplayed_df = pd.concat([matesplayed_df, pd.DataFrame([mateplayed_count])])
    winmates_df = pd.concat([winmates_df, pd.DataFrame([winmate_count])])
    mates_attack_df = pd.concat([mates_attack_df, pd.DataFrame([mates_attack_count])])
    winmates_attack_df = pd.concat([winmates_attack_df, pd.DataFrame([winmates_attack_count])])
    closematches_df = pd.concat([closematches_df, pd.DataFrame([closematches_count])])
    closewins_df = pd.concat([closewins_df, pd.DataFrame([closewins_count])])
    closewinsplayed_df = pd.concat([closewinsplayed_df, pd.DataFrame([closewinsplayed_count])])

# Transpose to match dimesnions with xarray dimensions (x: player, y: teammate)
matesplayed_df = matesplayed_df.transpose() # necessary if matrix is not symmetric
#closematches_df = closematches_df.transpose()
#closewins_df = closewins_df.transpose()

# Set index to teammate name
mates_df = mates_df.set_index(players_names)
winmates_df = winmates_df.set_index(players_names)
mates_attack_df = mates_attack_df.set_index(players_names)
winmates_attack_df = winmates_attack_df.set_index(players_names)
closematches_df = closematches_df.set_index(players_names)
closewins_df = closewins_df.set_index(players_names)
closewinsplayed_df = closewinsplayed_df.set_index(players_names)

# Convert to float (problems with nan)
#closematches_df = closematches_df.astype(int) # no sé per què em sortia error si no faig aquesta línia
closewins_df = closewins_df.astype(float)
closewinsplayed_df = closewinsplayed_df.astype(float)

# Create win / played ratio for each team (substitute 0 in the denominator by NaN, then recover 0 in the result)
winmatesplayed_df = winmates_df.div(mates_df.replace(0, pd.NA))


C:\Users\Admin\AppData\Local\Temp\ipykernel_6628\2264483374.py:113: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  matesplayed_df = pd.concat([matesplayed_df, pd.DataFrame([mateplayed_count])])
C:\Users\Admin\AppData\Local\Temp\ipykernel_6628\2264483374.py:119: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  closewinsplayed_df = pd.concat([closewinsplayed_df, pd.DataFrame([closewinsplayed_count])])


In [7]:
mates_df

,Antía,Dani,Elena,Gisela,Guille,Luis,Maider,Pablo,Pau,Pedro,Rebeca,Simone,Tiziano,Víctor
Antía,NaN,27,1,2,7,10,5,1,11,1,1,NaN,1,11
Dani,27,NaN,NaN,1,8,13,12,NaN,11,5,NaN,NaN,2,16
Elena,1,NaN,NaN,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN,NaN,4
Gisela,2,1,NaN,NaN,3,1,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN
Guille,7,8,NaN,3,NaN,3,4,NaN,12,NaN,NaN,NaN,1,4
Luis,10,13,2,1,3,NaN,9,1,10,6,2,4,2,10
Maider,5,12,2,NaN,4,9,NaN,NaN,5,2,2,NaN,NaN,4
Pablo,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
Pau,11,11,NaN,3,12,10,5,NaN,NaN,1,NaN,NaN,NaN,4
Pedro,1,5,NaN,NaN,NaN,6,2,NaN,1,NaN,NaN,NaN,NaN,6


In [8]:
# Creem una DataArray de xarray. Hi especifiquem els noms de cada dimensió
mates_da = xr.DataArray(mates_df.values, dims = ('teammate', 'player'),
                                      coords = {'teammate': mates_df.index, 'player': mates_df.columns})
matesplayed_da = xr.DataArray(matesplayed_df.values, dims = ('teammate', 'player'),
                                      coords = {'teammate': mates_df.index, 'player': mates_df.columns})
winmates_da = xr.DataArray(winmates_df.values, dims = ('teammate', 'player'),
                                      coords = {'teammate': winmates_df.index, 'player': winmates_df.columns})
winmatesplayed_da = xr.DataArray(winmatesplayed_df.values, dims = ('teammate', 'player'),
                                      coords = {'teammate': winmatesplayed_df.index, 'player': winmatesplayed_df.columns})
closewins_da = xr.DataArray(closewins_df.values, dims = ('teammate', 'player'),
                                      coords = {'teammate': winmatesplayed_df.index, 'player': winmatesplayed_df.columns})
closematches_da = xr.DataArray(closematches_df.values, dims = ('teammate', 'player'),
                                      coords = {'teammate': closematches_df.index, 'player': closematches_df.columns})
closewinsplayed_da = xr.DataArray(closewinsplayed_df.values, dims = ('teammate', 'player'),
                                      coords = {'teammate': closewinsplayed_df.index, 'player': closewinsplayed_df.columns})

# Combinem tots els DataArrays a un únic Dataset de xarray (cal que les coords siguin les mateixes per a tots)
dataset = xr.Dataset({"Teammates": mates_da,
                      "TeammatesPlayed": matesplayed_da,
                      "TeammatesWins": winmates_da,
                      "TeammatesWinsPlayed": winmatesplayed_da,
                      "CloseWins": closewins_da,
                      "CloseMatches": closematches_da,
                      "CloseWinsPlayed": closewinsplayed_da})

# TODO: el procés de crear el DataArray a partir del DataFrame es pot automatitzar amb una funció que faci un concat al dataframe. 

# Sote dataset
dataset.to_netcdf('teammates.nc', mode='w')

# dataset['goals'] = goals_da # si volem afegir un nou element
dataset

<xarray.Dataset> Size: 11kB
Dimensions:              (teammate: 14, player: 14)
Coordinates:
  * teammate             (teammate) object 112B 'Antía' 'Dani' ... 'Víctor'
  * player               (player) object 112B 'Antía' 'Dani' ... 'Víctor'
Data variables:
    Teammates            (teammate, player) object 2kB nan 27 1 ... nan nan nan
    TeammatesPlayed      (teammate, player) float64 2kB nan 0.2842 ... nan nan
    TeammatesWins        (teammate, player) object 2kB nan 18 0 ... nan nan nan
    TeammatesWinsPlayed  (teammate, player) object 2kB nan ... nan
    CloseWins            (teammate, player) float64 2kB nan 3.0 0.0 ... nan nan
    CloseMatches         (teammate, player) object 2kB nan 8 1 ... nan nan nan
    CloseWinsPlayed      (teammate, player) float64 2kB nan 0.375 ... nan nan